In [1]:
from datetime import datetime
from pyspark import SparkContext, HiveContext
from pyspark.sql import SparkSession, SQLContext
import pyspark.sql.functions as funcs
import pandas as pd
from pyspark.sql.functions import row_number,lit
from pyspark.sql.window import Window
import pyspark

In [2]:
spark = (SparkSession.builder.appName("pyspark-03-{}".format(datetime.today()))
        .master("spark://spark-master:7077")
        .getOrCreate())
sqlContext = SQLContext(spark)
#spark.sparkContext.getConf().getAll()

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
spark.sql("show tables").toPandas()

,namespace,tableName,isTemporary


In [4]:
df_orders = spark.read.format("parquet").load("s3a://pp-transaction/pp_transaction_20230530_parquet_data.parquet", header=True)
df_orders.limit(10).toPandas()

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,tps,data_time,bank_send,bank_receive,proxy_type,datatype
0,4,2023-05-30 07:20:02,KTB,SCB,NATID,TPS_SWT_LEG1_LEG2_byBANK_andPROXY
1,3,2023-05-30 07:20:02,TMB,SCB,ACCOUNT,TPS_SWT_LEG1_LEG2_byBANK_andPROXY
2,3,2023-05-30 07:20:02,UOBT,SCB,ACCOUNT,TPS_SWT_LEG1_LEG2_byBANK_andPROXY
3,1,2023-05-30 07:20:03,BAAC,SCB,MSISDN,TPS_SWT_LEG1_LEG2_byBANK_andPROXY
4,9,2023-05-30 07:20:03,BBL,SCB,ACCOUNT,TPS_SWT_LEG1_LEG2_byBANK_andPROXY
5,1,2023-05-30 07:20:03,BBL,SCB,MSISDN,TPS_SWT_LEG1_LEG2_byBANK_andPROXY
6,1,2023-05-30 07:20:03,GSB,SCB,BILLERID,TPS_SWT_LEG1_LEG2_byBANK_andPROXY
7,35,2023-05-30 07:20:03,KBNK,SCB,ACCOUNT,TPS_SWT_LEG1_LEG2_byBANK_andPROXY
8,6,2023-05-30 07:20:03,KBNK,SCB,BILLERID,TPS_SWT_LEG1_LEG2_byBANK_andPROXY
9,16,2023-05-30 07:20:03,KBNK,SCB,EWALLET,TPS_SWT_LEG1_LEG2_byBANK_andPROXY


In [5]:
df_orders = df_orders.withColumn("date", funcs.to_date("data_time"))
(
df_orders.write.mode("overwrite")
    .partitionBy("date")
    .option("compression", "snappy")
    .option("path", "s3a://warehouse/pp_transaction.delta")
    .format("delta")
    .saveAsTable("pp_transaction")
)

In [6]:
spark.sql("show databases").toPandas()

,namespace
0,default


In [7]:
spark.sql("show tables").toPandas()

,namespace,tableName,isTemporary
0,default,pp_transaction,False


In [8]:
pp = spark.sql("SELECT * FROM pp_transaction")
print(pp.count())

8763175


In [9]:
pp.limit(5).show()

+---+-------------------+---------+------------+----------+--------------------+----------+
|tps|          data_time|bank_send|bank_receive|proxy_type|            datatype|      date|
+---+-------------------+---------+------------+----------+--------------------+----------+
|  4|2023-05-30 07:20:02|      KTB|         SCB|     NATID|TPS_SWT_LEG1_LEG2...|2023-05-30|
| 70|2023-05-30 10:22:24|      SCB|        KBNK|   ACCOUNT|TPS_SWT_LEG1_LEG2...|2023-05-30|
|  3|2023-05-30 07:20:02|      TMB|         SCB|   ACCOUNT|TPS_SWT_LEG1_LEG2...|2023-05-30|
|  2|2023-05-30 10:22:24|      SCB|        KBNK|   EWALLET|TPS_SWT_LEG1_LEG2...|2023-05-30|
|  3|2023-05-30 07:20:02|     UOBT|         SCB|   ACCOUNT|TPS_SWT_LEG1_LEG2...|2023-05-30|
+---+-------------------+---------+------------+----------+--------------------+----------+



In [10]:
#df_orders1 = spark.read.format("parquet").load("s3a://pp-transaction/pp_transaction_20230531_parquet_data.parquet", header=True)
#df_orders1.limit(5).toPandas()

In [11]:
#df_orders1 = df_orders1.withColumn("date", funcs.to_date("data_time"))
#(
#df_orders1.write.mode("append")
#    .option("compression", "snappy")
#    .option("path", "s3a://warehouse/pp_transaction.delta")
#    .format("delta")
#    .saveAsTable("pp_transaction")
#)

In [12]:
#pp = spark.sql("SELECT * FROM pp_transaction")
#print(pp.count())

# Use case 1 check dup and check null data

In [13]:
temp = spark.sql("SELECT * FROM pp_transaction")
temp = temp.withColumn("date", funcs.to_date("data_time"))
(
temp.write.mode("overwrite")
    .partitionBy("date")
    .option("compression", "snappy")
    .option("path", "s3a://warehouse/use_case.delta")
    .format("delta")
    .saveAsTable("use_case")
)

In [14]:
spark.sql("show tables").toPandas()

,namespace,tableName,isTemporary
0,default,pp_transaction,False
1,default,use_case,False


In [15]:
use_case = spark.sql("SELECT * FROM use_case")
print(use_case.count())

8763175


In [16]:
dup = use_case.groupBy(use_case.columns).count().where(funcs.col('count') > 1)
print(dup.count())

0


In [17]:
dup1 = spark.sql("""SELECT COUNT(*) AS duplicate_count
FROM (
  SELECT *
  FROM use_case
  GROUP BY tps,data_time,bank_send,bank_receive,proxy_type,datatype,date
  HAVING COUNT(*) > 1
) AS duplicates;
""")

In [18]:
print(dup1.count())

1


In [19]:
use_case1 = spark.sql("SELECT * FROM use_case WHERE data_time IS NULL OR tps IS NULL OR bank_send IS NULL OR bank_receive IS NULL OR proxy_type IS NULL OR datatype IS NULL")
print(use_case1.count())

0


# use case 2 create table for bank and join to transaction table

In [20]:
bank_name = spark.sql("SELECT DISTINCT bank_send FROM use_case")
bn = bank_name.toPandas()

In [21]:
w = Window().orderBy(lit("A"))
bank_name = bank_name.withColumn("id",row_number().over(w))

In [22]:
(
bank_name.write.mode("overwrite")
    .option("compression", "snappy")
    .option("path", "s3a://warehouse/bank_name.delta")
    .format("delta")
    .saveAsTable("bank_name")
)

In [23]:
use_case2 = spark.sql("SELECT * FROM pp_transaction")
use_case2 = use_case2.withColumn("date", funcs.to_date("data_time"))
(
use_case2.write.mode("overwrite")
    .partitionBy("date")
    .option("compression", "snappy")
    .option("path", "s3a://warehouse/use_case2.delta")
    .format("delta")
    .saveAsTable("use_case2")
)

In [24]:
spark.sql("show tables").toPandas()

,namespace,tableName,isTemporary
0,default,bank_name,False
1,default,pp_transaction,False
2,default,use_case,False
3,default,use_case2,False


In [25]:
bank_name_df = spark.sql("SELECT * FROM bank_name")
use_case2_1 = spark.sql("SELECT * FROM use_case2")

In [26]:
temp = use_case2_1.join(bank_name_df,use_case2_1.bank_send == bank_name_df.bank_send).select(use_case2_1["*"],bank_name_df["id"])

In [27]:
temp.limit(5).show()

+---+-------------------+---------+------------+----------+--------------------+----------+---+
|tps|          data_time|bank_send|bank_receive|proxy_type|            datatype|      date| id|
+---+-------------------+---------+------------+----------+--------------------+----------+---+
| 15|2023-05-30 19:09:32|      GSB|         SCB|   ACCOUNT|TPS_SWT_LEG1_LEG2...|2023-05-30|  3|
|  1|2023-05-30 17:59:12|      BBL|        KBNK|     NATID|TPS_SWT_LEG1_LEG2...|2023-05-30| 13|
|  2|2023-05-30 15:36:20|     BAAC|        KBNK|   ACCOUNT|TPS_SWT_LEG1_LEG2...|2023-05-30| 26|
|  2|2023-05-30 20:21:05|      BAY|         KTB|    MSISDN|TPS_SWT_LEG1_LEG2...|2023-05-30| 16|
|  1|2023-05-30 19:09:32|      GSB|         SCB|  BILLERID|TPS_SWT_LEG1_LEG2...|2023-05-30|  3|
+---+-------------------+---------+------------+----------+--------------------+----------+---+



In [28]:
temp.printSchema()

root
 |-- tps: long (nullable = true)
 |-- data_time: timestamp (nullable = true)
 |-- bank_send: string (nullable = true)
 |-- bank_receive: string (nullable = true)
 |-- proxy_type: string (nullable = true)
 |-- datatype: string (nullable = true)
 |-- date: date (nullable = true)
 |-- id: integer (nullable = true)



In [29]:
use_case2_1.printSchema()

root
 |-- tps: long (nullable = true)
 |-- data_time: timestamp (nullable = true)
 |-- bank_send: string (nullable = true)
 |-- bank_receive: string (nullable = true)
 |-- proxy_type: string (nullable = true)
 |-- datatype: string (nullable = true)
 |-- date: date (nullable = true)



In [30]:
(
temp.write.mode("overwrite")
    .option("overwriteSchema", "True")
    .option("compression", "snappy")
    .option("path", "s3a://warehouse/use_case2.delta")
    .partitionBy("date")
    .format("delta")
    .saveAsTable("use_case2")
)

In [31]:
spark.sql("show tables").toPandas()

,namespace,tableName,isTemporary
0,default,bank_name,False
1,default,pp_transaction,False
2,default,use_case,False
3,default,use_case2,False


In [32]:
use_case2_2 = spark.sql("SELECT * FROM use_case2")
use_case2_2.limit(5).show()

+---+-------------------+---------+------------+----------+--------------------+----------+---+
|tps|          data_time|bank_send|bank_receive|proxy_type|            datatype|      date| id|
+---+-------------------+---------+------------+----------+--------------------+----------+---+
| 15|2023-05-30 19:09:32|      GSB|         SCB|   ACCOUNT|TPS_SWT_LEG1_LEG2...|2023-05-30|  3|
|  6|2023-05-30 22:24:37|      KTB|        KBNK|    MSISDN|TPS_SWT_LEG1_LEG2...|2023-05-30| 15|
|  1|2023-05-30 17:59:12|      BBL|        KBNK|     NATID|TPS_SWT_LEG1_LEG2...|2023-05-30| 13|
|  2|2023-05-30 13:34:16|      SCB|         GSB|    MSISDN|TPS_SWT_LEG1_LEG2...|2023-05-30|  5|
|  2|2023-05-30 15:36:20|     BAAC|        KBNK|   ACCOUNT|TPS_SWT_LEG1_LEG2...|2023-05-30| 26|
+---+-------------------+---------+------------+----------+--------------------+----------+---+



# 3.create new table from dedup and drop null from old table

In [33]:
use_case3 = spark.sql("""SELECT 
DISTINCT tps, data_time, bank_send, bank_receive, proxy_type, datatype, date FROM pp_transaction
WHERE tps IS NOT NULL AND data_time IS NOT NULL AND bank_send IS NOT NULL AND 
bank_receive IS NOT NULL AND proxy_type IS NOT NULL AND datatype IS NOT NULL""")


In [34]:
use_case3.limit(5).show()

+---+-------------------+---------+------------+----------+--------------------+----------+
|tps|          data_time|bank_send|bank_receive|proxy_type|            datatype|      date|
+---+-------------------+---------+------------+----------+--------------------+----------+
|  1|2023-05-30 07:00:00|     BAAC|         BBL|   ACCOUNT|TPS_SWT_LEG1_LEG2...|2023-05-30|
|  1|2023-05-30 07:00:00|     BAAC|         KTB|    MSISDN|TPS_SWT_LEG1_LEG2...|2023-05-30|
|  1|2023-05-30 07:00:00|      BAY|        KBNK|    MSISDN|TPS_SWT_LEG1_LEG2...|2023-05-30|
|  1|2023-05-30 07:00:00|      BBL|         KTB|     NATID|TPS_SWT_LEG1_LEG2...|2023-05-30|
|  1|2023-05-30 07:00:00|      BBL|         SCB|     NATID|TPS_SWT_LEG1_LEG2...|2023-05-30|
+---+-------------------+---------+------------+----------+--------------------+----------+



# 4. compare old and new table

In [35]:
use_case3_1 = spark.sql("SELECT * FROM pp_transaction")

In [36]:
use_case3_1.limit(5).show()

+---+-------------------+---------+------------+----------+--------------------+----------+
|tps|          data_time|bank_send|bank_receive|proxy_type|            datatype|      date|
+---+-------------------+---------+------------+----------+--------------------+----------+
|  4|2023-05-30 07:20:02|      KTB|         SCB|     NATID|TPS_SWT_LEG1_LEG2...|2023-05-30|
| 70|2023-05-30 10:22:24|      SCB|        KBNK|   ACCOUNT|TPS_SWT_LEG1_LEG2...|2023-05-30|
|  3|2023-05-30 07:20:02|      TMB|         SCB|   ACCOUNT|TPS_SWT_LEG1_LEG2...|2023-05-30|
|  2|2023-05-30 10:22:24|      SCB|        KBNK|   EWALLET|TPS_SWT_LEG1_LEG2...|2023-05-30|
|  3|2023-05-30 07:20:02|     UOBT|         SCB|   ACCOUNT|TPS_SWT_LEG1_LEG2...|2023-05-30|
+---+-------------------+---------+------------+----------+--------------------+----------+



In [37]:
print(use_case3.count())
print(use_case3_1.count())

8763175
8763175


In [38]:
use_case3.summary().show()

+-------+-----------------+---------+------------+----------+--------------------+
|summary|              tps|bank_send|bank_receive|proxy_type|            datatype|
+-------+-----------------+---------+------------+----------+--------------------+
|  count|          8763175|  8763175|     8763175|   8763175|             8763175|
|   mean|5.363574731761034|     null|        null|      null|                null|
| stddev|9.111288017508775|     null|        null|      null|                null|
|    min|                1|     BAAC|        BAAC|   ACCOUNT|TPS_SWT_LEG1_LEG2...|
|    25%|                1|     null|        null|      null|                null|
|    50%|                2|     null|        null|      null|                null|
|    75%|                6|     null|        null|      null|                null|
|    max|              169|     UOBT|        UOBT|     NATID|TPS_SWT_LEG1_LEG2...|
+-------+-----------------+---------+------------+----------+--------------------+



In [39]:
use_case3_1.summary().show()

+-------+-----------------+---------+------------+----------+--------------------+
|summary|              tps|bank_send|bank_receive|proxy_type|            datatype|
+-------+-----------------+---------+------------+----------+--------------------+
|  count|          8763175|  8763175|     8763175|   8763175|             8763175|
|   mean|5.363574731761034|     null|        null|      null|                null|
| stddev|9.111288017508743|     null|        null|      null|                null|
|    min|                1|     BAAC|        BAAC|   ACCOUNT|TPS_SWT_LEG1_LEG2...|
|    25%|                1|     null|        null|      null|                null|
|    50%|                2|     null|        null|      null|                null|
|    75%|                6|     null|        null|      null|                null|
|    max|              169|     UOBT|        UOBT|     NATID|TPS_SWT_LEG1_LEG2...|
+-------+-----------------+---------+------------+----------+--------------------+



In [40]:
use_case3.exceptAll(use_case3_1).show()

+---+---------+---------+------------+----------+--------+----+
|tps|data_time|bank_send|bank_receive|proxy_type|datatype|date|
+---+---------+---------+------------+----------+--------+----+
+---+---------+---------+------------+----------+--------+----+



In [41]:
spark.sql("show tables").toPandas()

,namespace,tableName,isTemporary
0,default,bank_name,False
1,default,pp_transaction,False
2,default,use_case,False
3,default,use_case2,False


In [42]:
data_transform = spark.sql("SELECT * FROM pp_transaction")

In [43]:
data_transform.limit(5).show()

+---+-------------------+---------+------------+----------+--------------------+----------+
|tps|          data_time|bank_send|bank_receive|proxy_type|            datatype|      date|
+---+-------------------+---------+------------+----------+--------------------+----------+
|  4|2023-05-30 07:20:02|      KTB|         SCB|     NATID|TPS_SWT_LEG1_LEG2...|2023-05-30|
| 70|2023-05-30 10:22:24|      SCB|        KBNK|   ACCOUNT|TPS_SWT_LEG1_LEG2...|2023-05-30|
|  3|2023-05-30 07:20:02|      TMB|         SCB|   ACCOUNT|TPS_SWT_LEG1_LEG2...|2023-05-30|
|  2|2023-05-30 10:22:24|      SCB|        KBNK|   EWALLET|TPS_SWT_LEG1_LEG2...|2023-05-30|
|  3|2023-05-30 07:20:02|     UOBT|         SCB|   ACCOUNT|TPS_SWT_LEG1_LEG2...|2023-05-30|
+---+-------------------+---------+------------+----------+--------------------+----------+



In [44]:
data_transform = data_transform.withColumn("year", funcs.year("data_time"))
data_transform = data_transform.withColumn("month", funcs.month("data_time"))
data_transform = data_transform.withColumn("day", funcs.dayofmonth("data_time"))
data_transform = data_transform.withColumn("hour", funcs.hour("data_time"))
data_transform = data_transform.withColumn("minute", funcs.minute("data_time"))
data_transform = data_transform.withColumn("second", funcs.second("data_time"))

In [45]:
data_transform.limit(5).toPandas()

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,tps,data_time,bank_send,bank_receive,proxy_type,datatype,date,year,month,day,hour,minute,second
0,4,2023-05-30 07:20:02,KTB,SCB,NATID,TPS_SWT_LEG1_LEG2_byBANK_andPROXY,2023-05-30,2023,5,30,7,20,2
1,70,2023-05-30 10:22:24,SCB,KBNK,ACCOUNT,TPS_SWT_LEG1_LEG2_byBANK_andPROXY,2023-05-30,2023,5,30,10,22,24
2,3,2023-05-30 07:20:02,TMB,SCB,ACCOUNT,TPS_SWT_LEG1_LEG2_byBANK_andPROXY,2023-05-30,2023,5,30,7,20,2
3,2,2023-05-30 10:22:24,SCB,KBNK,EWALLET,TPS_SWT_LEG1_LEG2_byBANK_andPROXY,2023-05-30,2023,5,30,10,22,24
4,3,2023-05-30 07:20:02,UOBT,SCB,ACCOUNT,TPS_SWT_LEG1_LEG2_byBANK_andPROXY,2023-05-30,2023,5,30,7,20,2


In [46]:
(
data_transform.write.mode("overwrite")
    .option("overwriteSchema", "True")
    .option("compression", "snappy")
    .option("path", "s3a://warehouse/data_transform.delta")
    .partitionBy("date")
    .format("delta")
    .saveAsTable("data_transform")
)

In [47]:
spark.sql("show tables").toPandas()

,namespace,tableName,isTemporary
0,default,bank_name,False
1,default,data_transform,False
2,default,pp_transaction,False
3,default,use_case,False
4,default,use_case2,False


In [48]:
spark.stop()